In [1]:
# imports
import pandas as pd
import numpy as np
#import seborn
from sklearn.preprocessing import MinMaxScaler, KBinsDiscretizer


In [2]:
# import data
data = pd.read_csv("accepted_2007_to_2018Q4.csv", engine='c')
data = data.drop(['id', 'member_id', 'settlement_term','settlement_percentage', 'settlement_amount', 'settlement_date','settlement_status', 'debt_settlement_flag_date', 'hardship_last_payment_amount', 'hardship_payoff_balance_amount', 'orig_projected_additional_accrued_interest',
           'hardship_loan_status', 'hardship_dpd', 'hardship_length','payment_plan_start_date','hardship_end_date', 'hardship_start_date', 'hardship_amount', 'deferral_term', 'hardship_status', 'hardship_reason', 'hardship_type',
           'sec_app_mths_since_last_major_derog', 'sec_app_collections_12_mths_ex_med', 'sec_app_chargeoff_within_12_mths', 'sec_app_num_rev_accts', 'sec_app_open_act_il', 'sec_app_revol_util', 'sec_app_open_acc', 'sec_app_mort_acc',
           'sec_app_inq_last_6mths', 'sec_app_earliest_cr_line', 'sec_app_fico_range_high', 'sec_app_fico_range_low', 'verification_status_joint', 'dti_joint', 'annual_inc_joint', 'desc', 'url', 'revol_bal_joint','mths_since_last_record', 'mths_since_recent_bc_dlq', 'mths_since_last_major_derog', 'mths_since_recent_revol_delinq', 'next_pymnt_d',
           'il_util', 'mths_since_rcnt_il' ], axis=1)

data = data.dropna()
#data
# maybe takeout these features as well:
# debt_settlement_flag

# I have left 'hardship_flag' included

# features removed on the second round of cleaning after looking at amount of missing rows in excel
#'mths_since_last_record', 'mths_since_recent_bc_dlq', 'mths_since_last_major_derog', 'mths_since_recent_revol_delinq', 'next_pymnt_d'

# idk what 'il_util' is but i removed it
data.convert_dtypes()
data.dtypes


C:\Users\zachs\AppData\Local\Temp\ipykernel_8232\646963163.py:2: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("accepted_2007_to_2018Q4.csv", engine='c')


loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
term                           object
int_rate                      float64
                               ...   
total_bc_limit                float64
total_il_high_credit_limit    float64
hardship_flag                  object
disbursement_method            object
debt_settlement_flag           object
Length: 104, dtype: object

In [3]:
data

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,3600.0,3600.0,3600.0,36 months,13.99,123.03,C,C4,leadman,10+ years,...,0.0,0.0,0.0,178050.0,7746.0,2400.0,13734.0,N,Cash,N
1,24700.0,24700.0,24700.0,36 months,11.99,820.28,C,C1,Engineer,10+ years,...,7.7,0.0,0.0,314017.0,39475.0,79300.0,24667.0,N,Cash,N
4,10400.0,10400.0,10400.0,60 months,22.45,289.91,F,F1,Contract Specialist,3 years,...,60.0,0.0,0.0,439570.0,95768.0,20300.0,88097.0,N,Cash,N
6,20000.0,20000.0,20000.0,36 months,9.17,637.58,B,B2,Vice President of Recruiting Operations,10+ years,...,100.0,0.0,0.0,388852.0,116762.0,31500.0,46452.0,N,Cash,N
7,20000.0,20000.0,20000.0,36 months,8.49,631.26,B,B1,road driver,10+ years,...,0.0,0.0,0.0,193390.0,27937.0,14500.0,36144.0,N,Cash,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260677,11200.0,11200.0,11200.0,60 months,13.49,257.66,C,C2,Sr. Photo Producer,5 years,...,33.3,0.0,2.0,23000.0,12581.0,15900.0,0.0,N,Cash,N
2260682,30000.0,30000.0,30000.0,60 months,13.99,697.90,C,C3,Fire Lieutenant,10+ years,...,66.7,0.0,0.0,107418.0,69005.0,29400.0,58418.0,N,Cash,N
2260692,16000.0,16000.0,16000.0,60 months,12.79,362.34,C,C1,Manager,10+ years,...,50.0,3.0,0.0,87473.0,65797.0,10100.0,73473.0,N,Cash,N
2260695,24000.0,24000.0,24000.0,60 months,10.49,515.74,B,B3,Database Administrator,10+ years,...,42.9,0.0,1.0,227883.0,140118.0,27900.0,172283.0,N,Cash,N


In [4]:

# apply normalization techniques 
df_sklearn = data.copy() 
for column  in df_sklearn.columns:
    if df_sklearn[column].dtype == np.float64:
        df_sklearn[column] = MinMaxScaler().fit_transform(np.array(df_sklearn[column]).reshape(-1,1)) 
display(df_sklearn) 

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,0.066667,0.066667,0.073202,36 months,0.338006,0.067410,C,C4,leadman,10+ years,...,0.000,0.000000,0.000000,0.017776,0.002953,0.002081,0.006481,N,Cash,N
1,0.607692,0.607692,0.610439,36 months,0.260125,0.474629,C,C1,Engineer,10+ years,...,0.077,0.000000,0.000000,0.031373,0.015050,0.071648,0.011641,N,Cash,N
4,0.241026,0.241026,0.246340,60 months,0.667445,0.164874,F,F1,Contract Specialist,3 years,...,0.600,0.000000,0.000000,0.043928,0.036512,0.018274,0.041575,N,Cash,N
6,0.487179,0.487179,0.490770,36 months,0.150312,0.367926,B,B2,Vice President of Recruiting Operations,10+ years,...,1.000,0.000000,0.000000,0.038856,0.044516,0.028406,0.021922,N,Cash,N
7,0.487179,0.487179,0.490770,36 months,0.123832,0.364235,B,B1,road driver,10+ years,...,0.000,0.000000,0.000000,0.019310,0.010651,0.013027,0.017057,N,Cash,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260677,0.261538,0.261538,0.266709,60 months,0.318536,0.146038,C,C2,Sr. Photo Producer,5 years,...,0.333,0.000000,0.032787,0.002270,0.004797,0.014293,0.000000,N,Cash,N
2260682,0.743590,0.743590,0.745385,60 months,0.338006,0.403155,C,C3,Fire Lieutenant,10+ years,...,0.667,0.000000,0.000000,0.010712,0.026309,0.026506,0.027569,N,Cash,N
2260692,0.384615,0.384615,0.388924,60 months,0.291277,0.207175,C,C1,Manager,10+ years,...,0.500,0.333333,0.000000,0.008718,0.025086,0.009046,0.034673,N,Cash,N
2260695,0.589744,0.589744,0.592616,60 months,0.201713,0.296767,B,B3,Database Administrator,10+ years,...,0.429,0.000000,0.016393,0.022759,0.053421,0.025149,0.081304,N,Cash,N


In [5]:
#discretise Data
for column  in df_sklearn.columns:
    if df_sklearn[column].dtype == np.float64:
        df_sklearn[column] = pd.cut(df_sklearn[column], bins= 10, labels= [0,1,2,3,4,5,6,7,8,9])
display(df_sklearn) 


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,disbursement_method,debt_settlement_flag
0,0,0,0,36 months,3,0,C,C4,leadman,10+ years,...,0,0,0,0,0,0,0,N,Cash,N
1,6,6,6,36 months,2,4,C,C1,Engineer,10+ years,...,0,0,0,0,0,0,0,N,Cash,N
4,2,2,2,60 months,6,1,F,F1,Contract Specialist,3 years,...,5,0,0,0,0,0,0,N,Cash,N
6,4,4,4,36 months,1,3,B,B2,Vice President of Recruiting Operations,10+ years,...,9,0,0,0,0,0,0,N,Cash,N
7,4,4,4,36 months,1,3,B,B1,road driver,10+ years,...,0,0,0,0,0,0,0,N,Cash,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2260677,2,2,2,60 months,3,1,C,C2,Sr. Photo Producer,5 years,...,3,0,0,0,0,0,0,N,Cash,N
2260682,7,7,7,60 months,3,4,C,C3,Fire Lieutenant,10+ years,...,6,0,0,0,0,0,0,N,Cash,N
2260692,3,3,3,60 months,2,2,C,C1,Manager,10+ years,...,4,3,0,0,0,0,0,N,Cash,N
2260695,5,5,5,60 months,2,2,B,B3,Database Administrator,10+ years,...,4,0,0,0,0,0,0,N,Cash,N
